# Natural landcover in floodplains

This indicator was made for the 2022 base blueprint.

In 2023 we made a change to add zero values to better depict the extent of the indicator

Created by Amy Keister, last run by Amy Keister on 21 June, 2023. It took 1 hour and 40 minutes to run.

In [122]:
import os
import arcpy

In [123]:
import time
start = time.time()

In [124]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\NaturalL\NaturalLan.gdb"

In [125]:
# define final indicator outputs
Out = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\NaturalL\NaturalLandcoverInFloodplains.tif"

In [126]:
# define rasters used for cell size, extent, and snapping
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [127]:
# define inputs
catch= r"F:\GIS_DATA\WaterResources\NHD\NHDPlusV21\NHDPlusV21_Catchment.gdb\CatchmentSECAS"
nlcd= r"F:\GIS_DATA\LanduseLandcover\NLCD\NLCD_landcover_2019_release_all_files_20210604\nlcd_2019_land_cover_l48_20210604\nlcd_2019_land_cover_l48_20210604.img"
FP = r"F:\GIS_DATA\WaterResources\Estimated_floodplain_CONUS\Estimated_floodplain_CONUS.tif"

### Start Analysis

In [128]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [129]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\NaturalL\NaturalLan.gdb


### Clip to floodplain

In [9]:
# Limit to the EPA floodplain
# it is important to make places outside the floodplain NODATA in this step
# so that we are only counting floodplain area inside each catchment
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Con(FP, nlcd, '',"Value = 1"); out_raster.save("nlcdFP")

In [10]:
# reclassify
out_raster = arcpy.sa.Reclassify("nlcdFP", "VALUE", "11 1;12 1;21 0;22 0;23 0;24 0;31 1;41 1;42 1;43 1;52 1;71 1;81 0;82 0;90 1;95 1", "DATA"); out_raster.save("nlcdFPrclss")

### zonal statistics

In [11]:
# zonal statistics. calculate mean natural cover in floodplain per catchment
out_raster = arcpy.sa.ZonalStatistics(catch, "GRIDCODE", "nlcdFPrclss", "MEAN", "DATA", "CURRENT_SLICE", 90); out_raster.save("ZStatNHDCatchNatFP")

In [12]:
# times
out_raster = arcpy.sa.Times("ZStatNHDCatchNatFP", 100); out_raster.save("ZStatNHDCatchNatFPx100")

In [13]:
# reclassify

# ArcGIS reclass includes the second value in the class for example,
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)

out_raster = arcpy.sa.Reclassify("ZStatNHDCatchNatFPx100", "VALUE", "0 60 1;60 70 2;70 80 3;80 90 4;90 100 5", "DATA"); out_raster.save("indi")

In [17]:
# Limit to the EPA floodplain. Have to do this again since my zonal stat was by catchment
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Con(FP, "indi", 0,"Value = 1"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [18]:
# set code block for next step
codeblock = """
def Reclass(v):
    if v == 5:
        return '5 = >90% natural habitat within the estimated floodplain, by catchment'
    elif v == 4:
        return '4 = >80-90%'
    elif v == 3:
        return '3 = >70-80%'
    elif v == 2:
        return '2 = >60-70%'
    elif v == 1:
        return '1 = ≤60% natural habitat within the estimated floodplain, by catchment'  
    elif v == 0:
        return '0 = Not identified as a floodplain'
"""

In [19]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("indicator", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\NaturalL\\NaturalLan.gdb\\indicator'>

In [134]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 5:
		return 9
	if Value == 4:
		return 61
	if Value == 3:
		return 112
	if Value == 2:
		return 152
	if Value == 1:
		return 182
	else:
		return 255
		
def Reclass2(Value):
	if Value == 5:
		return 13
	if Value == 4:
		return 77
	if Value == 3:
		return 144
	if Value == 2:
		return 199
	if Value == 1:
		return 239
	else:
		return 255
		
def Reclass3(Value):
	if Value == 5:
		return 94
	if Value == 4:
		return 138
	if Value == 3:
		return 161
	if Value == 2:
		return 173
	if Value == 1:
		return 182
	else:
		return 255
"""

In [135]:
# calculate Red field
arcpy.management.CalculateField("indicator", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("indicator", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("indicator", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'indicator'>

In [136]:
# clip to SE 2021 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("SEMask")

In [138]:
# export as .tif with SE extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("SEMask", Out, '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [24]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)

3865.747796535492
